# NSW Conservation and Sensitive Lists

## List Identifiers
* Conservation list: __[dr650](https://lists.ala.org.au/speciesListItem/list/dr650)__ (and [dr650 in test](https://lists-test.ala.org.au/speciesListItem/list/dr650))
* Sensitive list: __[dr487](https://lists.ala.org.au/speciesListItem/list/dr487)__ (and [dr18457 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18457))
* Collection: __[dr650](https://collections.ala.org.au/public/show/dr650)__ __[dr487](https://collections.ala.org.au/public/show/dr487)__
### Source Data

*  __[Home Page](https://data.bionet.nsw.gov.au)__
*  __[API/Data Info](https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames)__
*  __[Data Information and file downloads](https://www.environment.nsw.gov.au/research-and-publications/publications-search/bionet-sensitive-species-list)__


**Metadata Description**
Classification codes under the Threatened Species Conservation Act 1995: * Presumed Extinct * Critically Endangered * Endangered * Vulnerable * Endangered Population Learn more about the classifications used by the NSW Office of Environment & Heritage Machine-readable downloads can be obtained by searching NSW BioNet, the website for the Atlas of NSW Wildlife, and selecting: 1. Which species or group? = "all species", 2. Legal status = "Select records that fall under one or more categories" then choose "Threatened in NSW" 3. What area? = "Entire area" 4. Period of records? = "All records"

**Metadata URL**
https://data.bionet.nsw.gov.au


Import required libraries
Retrieve dataset from API
Set Project directory

In [1]:
import pandas as pd
import urllib.request, json
import os
import sys

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf


In [2]:
# %%script echo skipping # comment this line to download the files and save locally

with urllib.request.urlopen("https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames") as url:
    data = json.loads(url.read().decode())


In [3]:
data = pd.json_normalize(data, record_path =['value'])## Process Conservation

In [4]:
# df = data[data.stateConservation != 'Not Listed']
# df = data[data(['stateConservation'] != 'Not Listed') & (data['isCurrent'] = 'true')]
df= data[(data['stateConservation'] !='Not Listed') & (data['isCurrent'] == 'true')]

# df = data[(data['stateConservation'] != 'Not Listed') & (data['isCurrent'] = 'true')]

In [5]:
df['stateConservation'].unique()

array(['Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct'],
      dtype=object)

In [6]:
len(df.index)

1026

In [7]:
## Map statuses
state = 'NSW'
fname = statusDir + state + "-codes.csv"
df = lf.map_status(state, fname, df)

KeyError: 'sourceStatus'

Extract records where speciesID = taxonID
Map columns to DWC terms
Map status from stateConservation column

In [28]:
conservation = df[(df['speciesID'] == df['taxonID'])]
conservation = conservation[
    ['taxonRank', 'kingdom', 'class', 'order', 'family', 'genus', 'scientificName', 'specificEpithet', 'vernacularName',
     'establishmentMeans', 'stateConservation', 'protectedInNSW', 'sensitivityClass', 'TSProfileID',
     'countryConservation', 'dcterms_modified', 'speciesID', 'taxonID']]
conservation['status'] = conservation['stateConservation']
conservation = conservation.rename(columns={"stateConservation": "sourceStatus"})
conservation = conservation.rename(columns={"TSProfileID": "tsprofileid"})

Write dataframe to CSV - UTF-8 encoding

In [29]:
print("Writing to CSV")
conservation.to_csv(projectDir + "current-lists/conservation-lists/NSW-conservation-2023-03-15.csv",encoding="UTF-8",index=False)
print('Finished processing')

Writing to CSV
Finished processing


## Process Sensitive

In [8]:
df = data

Create dictionary for location obfuscation based on Category

In [9]:
dictn = {"Category 3": "1km",
       "Category 2": "10km",
       "Category 1": "WITHHOLD"
}

In [10]:
select = ['Category 3','Category 2','Category 1']

Set location generalisation values based on Category

In [11]:
df['generalisation'] = df['sensitivityClass']
df['generalisation'] = df['generalisation'].replace(dictn)

Extract records where speciesID equals taxonID and sensitivityClass is one of Category 3, Category 2 or Category1
Set status and sourceStatus

In [12]:
sensitive = df[(df['speciesID'] == df['taxonID']) & df['sensitivityClass'].isin(select)]
sensitive = sensitive[['taxonRank', 'kingdom' ,'class','order','family','genus','scientificName','specificEpithet','vernacularName','establishmentMeans','stateConservation','protectedInNSW','sensitivityClass','TSProfileID','countryConservation','dcterms_modified','speciesID','taxonID','generalisation']]
sensitive['status'] = sensitive['stateConservation']
sensitive = sensitive.rename(columns={"stateConservation": "sourceStatus", "sensitivityClass": "category", "TSProfileID": "tsprofileid"})

Write dataframe to CSV file - UTF-8 Encoding

In [14]:
print("Writing to CSV")
sensitive.to_csv(projectDir + "current-lists/sensitive-lists/NSW-sensitive-2023-04-27.csv",encoding="UTF-8",index=False)
print('Finished processing')

Writing to CSV
Finished processing


# Manual List check

**Instructions**
1. Load the lists above into the lists-test tool
2. Check the list name matching score and the text appearance on species pages
3. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
4. Save the production list into the `historical lists` directory by uncommenting the code section below.
5. Load the lists into production

### Conservation List - Download old and new and compare

In [40]:
# %%script echo skipping # comment this line to run this code

import datetime
monthStr = datetime.datetime.now().strftime('%Y%m')
ltype="C"
# conservation
filename = "NSW-conservation.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr650" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr650" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl, ltype)
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
174,Tympanocryptis lineata,Grassland Earless Dragon,Tympanocryptis lineata,Critically Endangered,added,NaN,NaN,NaN
291,Homoranthus floydii,None,Homoranthus floydii,Vulnerable,added,NaN,NaN,NaN
462,Persoonia oxycoccoides,None,Persoonia oxycoccoides,Endangered,added,NaN,NaN,NaN
838,Amytornis striatus striatus,Striated Grasswren (sandplain),Amytornis (Magnamytis) striatus striatus,Critically Endangered,added,NaN,NaN,NaN
1020,Amytornis striatus howei,Striated Grasswren,Amytornis striatus howei,Endangered,added,NaN,NaN,NaN
1021,Cormodes darwini,None,Cormodes darwini,Critically Endangered,added,NaN,NaN,NaN
89,Amytornis striatus,NaN,NaN,NaN,removed,Striated Grasswren,Amytornis (Magnamytis) striatus,Vulnerable
902,Tympanocryptis pinguicolla,NaN,NaN,NaN,removed,Grassland Earless Dragon,Tympanocryptis pinguicolla,Endangered


### Download Production list to Historical Lists directory

In [16]:
%%script echo skipping # comment this line to download the files and save locally
prodList = lf.download_ala_list(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading historical list')

Finished downloading historical list


### Sensitive List - Download old and new and compare

In [41]:

# %%script echo skipping # comment this line to run this code

import datetime
monthStr = datetime.datetime.now().strftime('%Y%m')
ltype = "S"
# conservation
filename = "NSW-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr487" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr18457" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl, ltype)
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)

changelist

,name,commonName_new,scientificName_new,listUpdate,commonName_old,scientificName_old
25,Pedionomus torquatus,Plains-wanderer,Pedionomus torquatus,added,NaN,NaN
98,Isotoma fluviatilis subsp. fluviatilis,None,Isotoma fluviatilis subsp. fluviatilis,added,NaN,NaN
120,Cyclodomorphus praealtus,Alpine She-oak Skink,Cyclodomorphus praealtus,added,NaN,NaN
7,Lathamus discolor,NaN,NaN,removed,Swift Parrot,Lathamus discolor


### Download Production lists to Historical List directory

In [43]:
# %%script echo skipping # comment this line to download the files and save locally
prodList = lf.download_ala_list(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)
print('Finished downloading historical list')

Finished downloading historical list
